## 1- Definition of arguments for function usage

In [1]:
import sys
import torch
import torch.optim as optim
import torchvision
from torchvision import transforms
from utils import *
import argparse
sys.argv = ['']

parser = argparse.ArgumentParser(description='Parameters training')
parser.add_argument('--model_architecture', type=str, default="VGG16", help='....')
parser.add_argument('--dataset', type=str, default="CIFAR10", help='....')
parser.add_argument('--batch_size', type=int, default=8, help='....')
parser.add_argument('--num_epochs', type=int, default=40, help='....')
parser.add_argument('--learning_rate', type=float, default=1e-3, help='....')
parser.add_argument('--optimizer_val', type=str, default="SGD", help='....')
parser.add_argument('--model_type', type=str, default="UNPRUNED", help='....')
parser.add_argument('--device', type=str, default=None, help='....')
parser.add_argument('--model_input', default=torch.ones((1, 3, 224, 224)), help='....')
parser.add_argument('--eval_metric', default="accuracy", help='....')
parser.add_argument('--pruning_seed', type=int, default=23, help='....')
parser.add_argument('--list_pruning', type=list, default = [0.6,0.6,0.53,0.53,0.4,0.4,0.4,0.5,0.5,0.5,0.6,0.6,0.6,0.5,0.5,0], help='....')

args = parser.parse_args()

if args.device is None:
    import torch
    args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# Modify for training and pruning
#args.model_architecture = "ResNet18"
#args.num_epochs = 40
args.dataset = "Tomato_Leaves"
args.eval_metric = "f1_score"

# Modify for pruning
args.list_pruning = [0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0]
type_pruning = 'HOMOGENEA'
base_percentage = 50

## Get Model, DATASET and TRAIN

In [4]:
train_loader, test_loader, num_classes = get_dataset(args)

In [3]:
model = get_model(args, num_classes)

In [5]:
train_model(args,
            train_loader = train_loader,
            test_loader = test_loader,
            model = model,
            num_classes = num_classes)

Epoch: [1/40]	 || Training Loss: 0.373	 || Val Loss: 0.154	 || Training F1-score: 0.863 	 ||  Val F1-score: 0.943 	 || Time: 00:07:12
Model Name: VGG16_Tomato_Leaves_UNPRUNED
Epoch: [2/40]	 || Training Loss: 0.073	 || Val Loss: 0.031	 || Training F1-score: 0.971 	 ||  Val F1-score: 0.989 	 || Time: 00:14:39
Model Name: VGG16_Tomato_Leaves_UNPRUNED
Epoch: [3/40]	 || Training Loss: 0.039	 || Val Loss: 0.028	 || Training F1-score: 0.985 	 ||  Val F1-score: 0.990 	 || Time: 00:22:05
Model Name: VGG16_Tomato_Leaves_UNPRUNED
Epoch: [4/40]	 || Training Loss: 0.029	 || Val Loss: 0.052	 || Training F1-score: 0.989 	 ||  Val F1-score: 0.982 	 || Time: 00:29:25
Epoch: [5/40]	 || Training Loss: 0.018	 || Val Loss: 0.116	 || Training F1-score: 0.992 	 ||  Val F1-score: 0.955 	 || Time: 00:36:50
Epoch: [6/40]	 || Training Loss: 0.016	 || Val Loss: 0.041	 || Training F1-score: 0.994 	 ||  Val F1-score: 0.985 	 || Time: 00:44:07
Epoch: [7/40]	 || Training Loss: 0.016	 || Val Loss: 0.038	 || Training F

## Pruning with multiple seeds

In [7]:
list_seeds = [23,42,97,112,167,214,256,333,425,512]

original_model_name = f'{args.model_architecture}_{args.dataset}_UNPRUNED'
for seed in list_seeds:
    #load original model
    model = torch.load(f'models/{original_model_name}.pth')
    args.seed = seed
    args.model_type = f'{type_pruning}_{base_percentage}_PRUNED_SEED_{seed}'
    #prune original model
    prune_model(model, args)
    args.model_type = f'{type_pruning}_{base_percentage}_PRUNED_FT_SEED_{seed}'
    #retraining pruned model
    train_model(args,
                train_loader = train_loader,
                test_loader = test_loader,
                model = model,
                num_classes = num_classes)

Epoch: [1/40]	 || Training Loss: 0.601	 || Val Loss: 0.333	 || Training F1-score: 0.756 	 ||  Val F1-score: 0.851 	 || Time: 00:02:46
Model Name: VGG16_Tomato_Leaves_HOMOGENEA_50_PRUNED_FT_SEED_23
Epoch: [2/40]	 || Training Loss: 0.183	 || Val Loss: 0.095	 || Training F1-score: 0.923 	 ||  Val F1-score: 0.962 	 || Time: 00:05:33
Model Name: VGG16_Tomato_Leaves_HOMOGENEA_50_PRUNED_FT_SEED_23
Epoch: [3/40]	 || Training Loss: 0.124	 || Val Loss: 0.098	 || Training F1-score: 0.950 	 ||  Val F1-score: 0.951 	 || Time: 00:08:22
Epoch: [4/40]	 || Training Loss: 0.083	 || Val Loss: 0.071	 || Training F1-score: 0.964 	 ||  Val F1-score: 0.975 	 || Time: 00:11:14
Model Name: VGG16_Tomato_Leaves_HOMOGENEA_50_PRUNED_FT_SEED_23


KeyboardInterrupt: 